# Importing Required Libraries

In [11]:
import torch
import cv2
from matplotlib import pyplot as plt
from pathlib import Path
import pandas as pd
import os
import sys
import pandas as pd
import logging

# Append the parent directory to the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../')))

# Now import PostgresConnection from the db_connection module
from db_connection.yolo_db_connection import PostgresConnection


# Loading YOLOv5 Model

In [2]:
# Load YOLOv5 model (you can use 'yolov5s' or other variations of the model)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # small version of YOLO


Using cache found in C:\Users\habteyes.asfaw/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-13 Python-3.12.7 torch-2.4.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


# Preparing the Image Data

In [3]:
# Path to the directory where images are stored
image_dir = Path('../data/photos')

In [4]:
# List all image files
image_files = list(image_dir.glob('*.jpg'))
print(f'Total images found: {len(image_files)}')

Total images found: 4821


 # Performing Object Detection on Images

In [ ]:
# Perform detection on each image and store results
detections = []
for img_path in image_files:
    # Perform inference on image
    results = model(img_path)
    
    # Save results (bounding boxes, labels, scores)
    detection_data = results.pandas().xyxy[0]  # convert detection to pandas DataFrame
    detection_data['image'] = img_path.name
    detections.append(detection_data)
    
    # Visualize the detection results
    # results.show()

# Combine all detection results into a single DataFrame
detection_df = pd.concat(detections, ignore_index=True)    




In [17]:
# Rename column in DataFrame before saving to SQL
detection_df.rename(columns={
    'xmin': 'xmin_value',
    'xmax': 'xmax_value'
}, inplace=True)

 # Processing Detection Results

In [18]:
# Display a few rows of the detection results
detection_df.head()

# Columns include: xmin, ymin, xmax, ymax, confidence, class, name (class label), image


,xmin_value,ymin,xmax_value,ymax,confidence,class,name,image
0,318.366425,394.813049,382.800201,462.255859,0.729678,32,sports ball,CheMed123_13.jpg
1,739.459229,544.979797,807.581909,615.879944,0.684087,32,sports ball,CheMed123_13.jpg
2,738.390198,542.634155,807.933777,616.783691,0.677431,41,cup,CheMed123_13.jpg
3,668.202393,589.573730,730.860596,657.322266,0.511538,32,sports ball,CheMed123_13.jpg
4,231.458694,473.585480,299.499542,536.707458,0.450486,32,sports ball,CheMed123_13.jpg


# Storing Detection Results in Database

In [19]:
from sqlalchemy import create_engine

# PostgreSQL connection parameters
dbname = "object_detection"
user = "postgres"
password = "root"
host = "localhost"
port = "5432"

# Create a connection string
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(connection_string)

# Save detection results to the PostgreSQL database
detection_df.to_sql('object_detection_results', engine, if_exists='replace', index=False)
logging.info("Detection results saved to PostgreSQL database.")


INFO:root:Detection results saved to PostgreSQL database.


In [22]:
import logging

logging.basicConfig(filename='object_detection.log', level=logging.INFO)

try:
    # Object detection process
    logging.info('Object detection completed successfully.')
except Exception as e:
    logging.error(f'Error occurred: {e}')


INFO:root:Object detection completed successfully.
